# ResNet-18 Pruning & Quantization (No Library)

In [2]:
import sys
sys.path.append('.')

import torch
import torch.nn as nn
from collections import OrderedDict
from models.resnet import ResNet18

# ==================== Configuration ====================
CKPT_PATH = './checkpoints/resnet18_ckpt_best.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==================== Load Model ====================
model = ResNet18()
ckpt = torch.load(CKPT_PATH, map_location='cpu')

# Handle DDP state_dict
state_dict = ckpt['net']
if list(state_dict.keys())[0].startswith('module.'):
    state_dict = OrderedDict([(k.replace('module.', ''), v) for k, v in state_dict.items()])
model.load_state_dict(state_dict)
model = model.to(DEVICE)
model.eval()

# ==================== Model Info ====================
print('=' * 60)
print('ResNet-18 for CIFAR-10 - Model Information')
print('=' * 60)
print(f"Checkpoint: {CKPT_PATH}")
print(f"Best Accuracy: {ckpt.get('acc', 'N/A'):.2f}%" if 'acc' in ckpt else f"Accuracy: N/A")
print(f"Trained Epochs: {ckpt.get('epoch', 'N/A')}")
print(f"Device: {DEVICE}")
print()

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model Size: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print()

# Layer-wise breakdown
print('Layer-wise Parameter Count:')
print('-' * 60)
conv_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        params = sum(p.numel() for p in module.parameters())
        conv_layers.append((name, module, params))
        print(f"{name:30s} | Conv2d | in={module.in_channels:4d}, out={module.out_channels:4d}, k={module.kernel_size[0]} | {params:,} params")
    elif isinstance(module, nn.Linear):
        params = sum(p.numel() for p in module.parameters())
        print(f"{name:30s} | Linear | in={module.in_features:4d}, out={module.out_features:4d}        | {params:,} params")

print()
print('=' * 60)
print(f"Total Conv2d Layers: {len(conv_layers)}")
print(f"Total Conv2d Parameters: {sum(p for _, _, p in conv_layers):,}")
print('=' * 60)

ResNet-18 for CIFAR-10 - Model Information
Checkpoint: ./checkpoints/resnet18_ckpt_best.pth
Best Accuracy: 95.43%
Trained Epochs: 182
Device: cuda

Total Parameters: 11,173,962
Trainable Parameters: 11,173,962
Model Size: 42.63 MB (FP32)

Layer-wise Parameter Count:
------------------------------------------------------------
conv1                          | Conv2d | in=   3, out=  64, k=3 | 1,728 params
layer1.0.conv1                 | Conv2d | in=  64, out=  64, k=3 | 36,864 params
layer1.0.conv2                 | Conv2d | in=  64, out=  64, k=3 | 36,864 params
layer1.1.conv1                 | Conv2d | in=  64, out=  64, k=3 | 36,864 params
layer1.1.conv2                 | Conv2d | in=  64, out=  64, k=3 | 36,864 params
layer2.0.conv1                 | Conv2d | in=  64, out= 128, k=3 | 73,728 params
layer2.0.conv2                 | Conv2d | in= 128, out= 128, k=3 | 147,456 params
layer2.0.shortcut.0            | Conv2d | in=  64, out= 128, k=1 | 8,192 params
layer2.1.conv1           